In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /gdrive

Mounted at /content/gdrive


In [ ]:
!ls "/content/gdrive/My Drive"

'Colab Notebooks'   NPZ_files   physionet


In [ ]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 14.5 MB/s 


In [ ]:
#import pyeeg
from tensorflow import keras 

In [ ]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [ ]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings 
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [ ]:
# localized fetch_data function by using mne library
# https://github.com/mne-tools/mne-python/blob/maint/0.20/mne/datasets/sleep_physionet/age.py#L18-L111
data_path = _data_path
BASE_URL = 'https://physionet.org/physiobank/database/sleep-edfx/sleep-cassette/'
def fetch_data(subjects, recording=[1, 2], path=None, force_update=False,
               update_path=None, base_url=BASE_URL,
               verbose=None):  # noqa: D301
    records = np.loadtxt(AGE_SLEEP_RECORDS,
                         skiprows=1,
                         delimiter=',',
                         usecols=(0, 1, 2, 6, 7),
                         dtype={'names': ('subject', 'record', 'type', 'sha',
                                          'fname'),
                                'formats': ('<i2', 'i1', '<S9', 'S40', '<S22')}
                         )
    psg_records = records[np.where(records['type'] == b'PSG')]
    hyp_records = records[np.where(records['type'] == b'Hypnogram')]

    path = data_path(path=path, update_path=update_path)
    params = [path, force_update, base_url]
    fnames = []
    for subject in subjects:
        for idx in np.where(psg_records['subject'] == subject)[0]:
            if psg_records['record'][idx] in recording:
                psg_fname = _fetch_one(psg_records['fname'][idx].decode(),
                                       psg_records['sha'][idx].decode(),
                                       *params)
                hyp_fname = _fetch_one(hyp_records['fname'][idx].decode(),
                                       hyp_records['sha'][idx].decode(),
                                       *params)
                fnames.append([psg_fname, hyp_fname])

    return fnames

In [ ]:
npz_files = sorted(glob.glob("/content/gdrive/My Drive/NPZ_files/***.npz"))

In [ ]:
X = np.zeros((0, 3000, 1))
y = []
for fn in tqdm(npz_files[:number_of_files]):
    samples = np.load(fn)
    X_data = samples['x']
    X = np.concatenate((X, X_data), axis=0)
    y.extend(samples['y'])
y = np.array(y)

#sorted(glob.glob("/gdrive/My Drive/NPZ_files/***.npz"))


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
npz_files

['/content/gdrive/My Drive/NPZ_files/SC4001E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4002E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4011E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4012E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4021E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4022E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4031E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4032E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4041E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4042E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4051E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4052E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4061E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4062E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4071E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4072E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4081E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4082E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4091E0.npz',
 '/content/g

In [ ]:
len(npz_files[:number_of_files])

100

In [ ]:
npz_files[:number_of_files]

['/content/gdrive/My Drive/NPZ_files/SC4001E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4002E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4011E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4012E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4021E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4022E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4031E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4032E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4041E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4042E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4051E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4052E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4061E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4062E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4071E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4072E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4081E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4082E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4091E0.npz',
 '/content/g

In [ ]:
npz_files[124:126]

['/content/gdrive/My Drive/NPZ_files/SC4661E0.npz',
 '/content/gdrive/My Drive/NPZ_files/SC4662E0.npz']

In [ ]:
X

array([[[ 8.11135578e+00],
        [ 1.74886456e+01],
        [ 2.12395611e+01],
        ...,
        [-1.03619051e+01],
        [-1.11120882e+01],
        [-2.10989022e+00]],

       [[-1.07369967e+01],
        [-1.13934069e+01],
        [-4.45421267e+00],
        ...,
        [ 5.88424911e+01],
        [ 4.83399277e+01],
        [ 5.36849823e+01]],

       [[ 6.13743591e+01],
        [ 3.86813202e+01],
        [ 4.95589752e+01],
        ...,
        [ 3.36175842e+01],
        [ 3.43677673e+01],
        [ 3.18358974e+01]],

       ...,

       [[ 3.47106218e+00],
        [-5.03052510e-02],
        [ 2.56556773e+00],
        ...,
        [ 9.55799758e-01],
        [-1.86129427e+00],
        [ 3.37045169e+00]],

       [[ 2.26373625e+00],
        [ 3.52136761e-01],
        [ 2.86739922e+00],
        ...,
        [ 2.54041519e+01],
        [ 2.02730160e+01],
        [ 1.82608051e+01]],

       [[ 1.96693535e+01],
        [ 1.65504265e+01],
        [ 1.54437122e+01],
        ...,
        

In [ ]:
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [ ]:
from keras.utils import np_utils

In [ ]:
if VBS:
    print("Shape of the input data: {}".format(X.shape))
    print("Shape of the sleep stages: {}".format(y.shape))


Shape of the input data: (116718, 3000, 1)
Shape of the sleep stages: (116718,)


In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)
    
    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)
    
    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)
    
    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

In [ ]:
def butter_bandpass(lowcut, highpass, fs, order=4):
    nyq = 0.5 * fs
    #       low = lowcut / nyq
    high = highpass / nyq
    b, a = butter(order, high, btype='highpass')
    return b, a

In [ ]:
def butter_bandpass_filter(data, highpass, fs, order=4):
    b, a = butter_bandpass(0, highpass, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
#npz_files = sorted(glob.glob(os.path.join(output_path, "*.npz")))
X_sub_466 = np.zeros((0, 3000, 1))
y_sub_466 = []
for fn in tqdm(npz_files[124:126]): #extracting data from the npz files corresponding to both tge recording 1, 2 of the 73rd subject (no 53rd subject)
    samples = np.load(fn)
    X_data = samples['x']
    X_sub_466 = np.concatenate((X_sub_466, X_data), axis=0)
    y_sub_466.extend(samples['y'])
y_sub_466 = np.array(y_sub_466)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
X_sub_466

array([[[ 19.33431053],
        [-37.49255371],
        [ -7.04297924],
        ...,
        [-16.205616  ],
        [ -6.21001244],
        [-35.36385727]],

       [[-20.64810753],
        [-39.15848541],
        [-16.205616  ],
        ...,
        [ 17.39072037],
        [ -8.06105042],
        [ -9.17167282]],

       [[-13.98437119],
        [  0.63882786],
        [  0.17606838],
        ...,
        [ 16.37265015],
        [-12.13333321],
        [  9.52380943]],

       ...,

       [[ 20.94554329],
        [ 20.19609261],
        [ 14.70012188],
        ...,
        [ 21.11208725],
        [ 11.20268631],
        [  5.87326002]],

       [[  8.70451736],
        [ 10.36996365],
        [  9.37069607],
        ...,
        [-48.00390625],
        [-43.0908432 ],
        [-35.26324844]],

       [[ -5.7015872 ],
        [-21.44004822],
        [  4.04126978],
        ...,
        [  3.37509155],
        [ 19.03028107],
        [ 35.51819229]]])

In [ ]:
y_sub_466

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [ ]:
pd.Series(y_sub_466).value_counts()

0    2096
2     836
1     610
4     360
3     118
dtype: int64

In [ ]:
if VBS:
    print("Shape of the subject(466) data: {}".format(X_sub_466.shape))
    print("Shape of the corresponding sleep stage: {}".format(y_sub_466.shape))

Shape of the subject(466) data: (4020, 3000, 1)
Shape of the corresponding sleep stage: (4020,)


In [ ]:
pp_x = np.array([butter_bandpass_filter(sample, highpass=40.0, fs=100, order=4) for sample in X])

if VBS:
    print(pp_x.shape)

(116718, 3000, 1)


In [ ]:
y.shape

(116718,)

In [ ]:
type(y)

numpy.ndarray

In [ ]:
pp_x

array([[[ 3.91319654e-02],
        [ 8.43712310e-02],
        [ 1.02466935e-01],
        ...,
        [-4.99893880e-02],
        [-5.36085289e-02],
        [-1.01788349e-02]],

       [[-5.17989585e-02],
        [-5.49657067e-02],
        [-2.14886513e-02],
        ...,
        [ 2.83876381e-01],
        [ 2.33208409e-01],
        [ 2.58994788e-01]],

       [[ 2.96090982e-01],
        [ 1.86611970e-01],
        [ 2.39089513e-01],
        ...,
        [ 1.62182769e-01],
        [ 1.65801910e-01],
        [ 1.53587300e-01]],

       ...,

       [[ 1.67455958e-02],
        [-2.42689804e-04],
        [ 1.23771796e-02],
        ...,
        [ 4.61110622e-03],
        [-8.97952265e-03],
        [ 1.62602162e-02]],

       [[ 1.09210409e-02],
        [ 1.69882864e-03],
        [ 1.38333184e-02],
        ...,
        [ 1.22558352e-01],
        [ 9.78039900e-02],
        [ 8.80963939e-02]],

       [[ 9.48917148e-02],
        [ 7.98449401e-02],
        [ 7.45057705e-02],
        ...,
        

In [ ]:
pp_x_rshp=pp_x.reshape(116718, 3000)

In [ ]:
pp_x_rshp_df=pd.DataFrame(pp_x_rshp)

In [ ]:
pp_x_rshp_df['sleep_stage']=y

In [ ]:
pp_x_rshp_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,sleep_stage
0,0.039132,0.084371,0.102467,0.033251,0.045465,0.026465,0.008369,0.001131,-0.006560,0.009274,...,-0.034156,0.001583,-0.057228,-0.041394,-0.023751,0.002488,-0.049989,-0.053609,-0.010179,0
1,-0.051799,-0.054966,-0.021489,-0.001131,-0.016512,-0.114229,-0.056775,-0.085728,-0.113324,-0.078490,...,0.264876,0.197922,0.267138,0.264876,0.312377,0.251304,0.283876,0.233208,0.258995,0
2,0.296091,0.186612,0.239090,0.259900,0.302877,0.316901,0.272567,0.281614,0.220541,0.265328,...,0.164445,0.146349,0.176207,0.176659,0.149516,0.163540,0.162183,0.165802,0.153587,0
3,0.134134,0.116491,0.116491,0.070347,0.066728,0.037322,0.030989,0.039584,0.023751,0.020131,...,-0.087086,-0.040942,0.009726,0.035060,0.129610,0.045465,0.126896,0.205612,0.274829,0
4,0.333640,0.359878,0.381141,0.386117,0.373450,0.372998,0.360783,0.367117,0.359878,0.382046,...,-0.121920,-0.060847,0.000679,0.060394,0.045465,0.084371,0.057680,0.122372,0.207422,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.117219,0.233710,0.088582,0.195365,0.141488,0.136634,0.103629,0.213810,0.167699,0.141488,...,-0.108482,-0.110424,-0.112365,-0.064798,-0.171096,-0.115278,-0.044898,-0.099746,-0.122073,0
116714,-0.087611,-0.076447,-0.056061,-0.058488,-0.075962,-0.053149,-0.065769,-0.104114,-0.086155,-0.121102,...,0.034219,0.093921,0.097319,0.070623,0.069167,0.040529,0.058488,0.046839,0.049266,0
116715,0.016746,-0.000243,0.012377,0.020629,0.014319,-0.042956,-0.085669,-0.076447,-0.066740,-0.060915,...,0.031792,0.027909,0.014804,0.017231,0.020143,0.001213,0.004611,-0.008980,0.016260,0
116716,0.010921,0.001699,0.013833,-0.000243,0.008494,-0.000243,0.002670,-0.001699,-0.003640,0.011406,...,0.126927,0.115763,0.124014,0.130810,0.130810,0.124014,0.122558,0.097804,0.088096,0


In [ ]:
pp_x_rshp_df.to_csv(r'/content/gdrive/My Drive/physionet/ensemble/50_subs/original_50_subs_dt_original_y_x_not_bootstraped.csv', header=False, index=False)

In [ ]:
dt_transfer_set=pp_x_rshp_df.sample(frac=1,replace=True, random_state=248)

In [ ]:
dt_transfer_set

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,sleep_stage
2302,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
31195,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
82207,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,1
2198,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
107738,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20894,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402,2
78249,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010,4
57039,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334,4
34319,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143,2


In [ ]:
#could save this to drive only after deleting transfer_set_50_subs.csv from drive (have a backup of this on pc/hard disc)

In [ ]:
dt_transfer_set.to_csv(r'/gdrive/My Drive/physionet/ensemble/50_subs/original_bootstrapped_50_subs_transfer_set.csv', header=False, index=False) #original_50_subs_transfer_set.csv

In [ ]:
x_dt_transfer=dt_transfer_set.drop('sleep_stage',axis=1)
y_dt_transfer=dt_transfer_set['sleep_stage']

In [ ]:
y_dt_transfer_=np_utils.to_categorical(y_dt_transfer)

In [ ]:
x_dt_transfer_rshp=np.array(x_dt_transfer).reshape(116718,3000,1)

In [ ]:
pp_X_sub_466 = np.array([butter_bandpass_filter(sample, highpass=40.0, fs=100, order=4) for sample in X_sub_466])

if VBS:
    print(pp_X_sub_466.shape)

(4020, 3000, 1)


In [ ]:
pp_X_sub_466

array([[[ 0.09327535],
        [-0.18087695],
        [-0.03397775],
        ...,
        [-0.07818146],
        [-0.02995923],
        [-0.17060739]],

       [[-0.09961356],
        [-0.18891398],
        [-0.07818146],
        ...,
        [ 0.08389881],
        [-0.03888928],
        [-0.0442473 ]],

       [[-0.06746541],
        [ 0.00308192],
        [ 0.00084941],
        ...,
        [ 0.07898729],
        [-0.05853537],
        [ 0.04594613]],

       ...,

       [[ 0.10104849],
        [ 0.09743289],
        [ 0.07091844],
        ...,
        [ 0.10185196],
        [ 0.05404561],
        [ 0.02833462]],

       [[ 0.04199358],
        [ 0.05002827],
        [ 0.04520746],
        ...,
        [-0.23158733],
        [-0.20788502],
        [-0.17012202]],

       [[-0.02750641],
        [-0.10343415],
        [ 0.01949647],
        ...,
        [ 0.0162826 ],
        [ 0.09180861],
        [ 0.17135196]]])

In [ ]:
y_sub_466_=np_utils.to_categorical(y_sub_466)

In [ ]:
y_sub_466_

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
pp_X_sub_466_rshp=pp_X_sub_466.reshape(4020, 3000)

In [ ]:
pp_X_sub_466_rshp_df=pd.DataFrame(pp_X_sub_466_rshp)

In [ ]:
pp_X_sub_466_rshp_df['sleep_stage']=y_sub_466

In [ ]:
pp_X_sub_466_rshp_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,sleep_stage
0,0.093275,-0.180877,-0.033978,0.000849,-0.034871,-0.049605,-0.036210,-0.154533,-0.082646,0.106670,...,0.001296,0.001742,0.090150,-0.022815,-0.059428,-0.107651,-0.078181,-0.029959,-0.170607,0
1,-0.099614,-0.188914,-0.078181,-0.035764,-0.118813,-0.025941,-0.074609,-0.088898,-0.105418,-0.028173,...,-0.037996,-0.084432,-0.098721,-0.040675,-0.061214,-0.095595,0.083899,-0.038889,-0.044247,0
2,-0.067465,0.003082,0.000849,0.005761,-0.160784,-0.325097,-0.310363,-0.283126,-0.271070,-0.086665,...,0.029426,0.097294,0.066932,0.064699,0.052197,0.115600,0.078987,-0.058535,0.045946,0
3,-0.097381,0.036123,-0.029513,-0.051838,-0.020136,-0.023262,-0.087558,-0.021029,-0.051391,-0.043801,...,0.053983,0.078541,0.112921,0.079880,0.052197,0.054876,0.053537,0.051751,0.147302,0
4,0.097740,0.166948,0.161144,0.273662,0.258035,0.262946,0.345103,0.166055,0.241068,0.243747,...,0.100419,0.173199,0.189273,0.225440,0.186148,0.169627,0.196417,0.158018,0.100419,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4015,0.230809,0.187421,0.236835,0.135598,0.129973,0.151265,0.097433,0.127563,0.107476,0.072124,...,0.020300,-0.054423,-0.009428,-0.007821,0.029942,-0.009428,0.004231,-0.012241,-0.016660,0
4016,-0.020677,-0.017061,-0.017463,0.024719,-0.007821,-0.023891,-0.026301,-0.016258,-0.013847,-0.081339,...,0.084577,0.071320,0.060473,0.012265,0.026728,0.033959,0.073731,0.068106,0.109083,0
4017,0.101048,0.097433,0.070918,0.067705,0.068106,0.066098,0.061277,0.016684,0.004231,0.036771,...,0.105066,0.109083,0.071722,0.070115,0.105468,0.068508,0.101852,0.054046,0.028335,0
4018,0.041994,0.050028,0.045207,0.037575,0.066499,0.019496,0.060072,0.114707,0.084577,0.024317,...,-0.035943,-0.031925,-0.035943,-0.096605,-0.197842,-0.185790,-0.231587,-0.207885,-0.170122,0


In [ ]:
pp_X_sub_466_rshp_df.to_csv(r'/content/gdrive/My Drive/physionet/ensemble/50_subs/subject_466_dataset.csv', header=False, index=False)

In [ ]:
pp_X_sub_466_rshp_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,sleep_stage
0,0.093275,-0.180877,-0.033978,0.000849,-0.034871,-0.049605,-0.036210,-0.154533,-0.082646,0.106670,...,0.001296,0.001742,0.090150,-0.022815,-0.059428,-0.107651,-0.078181,-0.029959,-0.170607,0
1,-0.099614,-0.188914,-0.078181,-0.035764,-0.118813,-0.025941,-0.074609,-0.088898,-0.105418,-0.028173,...,-0.037996,-0.084432,-0.098721,-0.040675,-0.061214,-0.095595,0.083899,-0.038889,-0.044247,0
2,-0.067465,0.003082,0.000849,0.005761,-0.160784,-0.325097,-0.310363,-0.283126,-0.271070,-0.086665,...,0.029426,0.097294,0.066932,0.064699,0.052197,0.115600,0.078987,-0.058535,0.045946,0
3,-0.097381,0.036123,-0.029513,-0.051838,-0.020136,-0.023262,-0.087558,-0.021029,-0.051391,-0.043801,...,0.053983,0.078541,0.112921,0.079880,0.052197,0.054876,0.053537,0.051751,0.147302,0
4,0.097740,0.166948,0.161144,0.273662,0.258035,0.262946,0.345103,0.166055,0.241068,0.243747,...,0.100419,0.173199,0.189273,0.225440,0.186148,0.169627,0.196417,0.158018,0.100419,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4015,0.230809,0.187421,0.236835,0.135598,0.129973,0.151265,0.097433,0.127563,0.107476,0.072124,...,0.020300,-0.054423,-0.009428,-0.007821,0.029942,-0.009428,0.004231,-0.012241,-0.016660,0
4016,-0.020677,-0.017061,-0.017463,0.024719,-0.007821,-0.023891,-0.026301,-0.016258,-0.013847,-0.081339,...,0.084577,0.071320,0.060473,0.012265,0.026728,0.033959,0.073731,0.068106,0.109083,0
4017,0.101048,0.097433,0.070918,0.067705,0.068106,0.066098,0.061277,0.016684,0.004231,0.036771,...,0.105066,0.109083,0.071722,0.070115,0.105468,0.068508,0.101852,0.054046,0.028335,0
4018,0.041994,0.050028,0.045207,0.037575,0.066499,0.019496,0.060072,0.114707,0.084577,0.024317,...,-0.035943,-0.031925,-0.035943,-0.096605,-0.197842,-0.185790,-0.231587,-0.207885,-0.170122,0


In [ ]:
import numpy as np
np.random.seed(999)

import tensorflow
tensorflow.random.set_seed(999)

import random
#random.seed(999)

In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-ilshgyii
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-ilshgyii
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=6d0a9bc212823d475a0b79ffa466e369e157daa4a432357831731037d4708a13
  Stored in directory: /tmp/pip-ephem-wheel-cache-f7k7sfo4/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
Fs = 100

In [ ]:
loaded_model_shuf1=model_b(verbose=VBS)
loaded_model_shuf2=model_b(verbose=VBS)
loaded_model_shuf3=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
#dt_shuf1
checkpoint_path_load_model_30_epochs_shuf1 = "/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf1_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf1)
#latest_30_epochs_shuf1_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf1_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf1)

/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt


In [ ]:
#dt_shuf2
checkpoint_path_load_model_30_epochs_shuf2 = "/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf2_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf2)
#latest_300_epochs_shuf2_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf2_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf2)

/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt


In [ ]:
#dt_shuf3
checkpoint_path_load_model_30_epochs_shuf3 = "/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf3_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf3)
#latest_300_epochs_shuf3_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf3_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf3)

/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt


In [ ]:
loaded_model_shuf1.load_weights(checkpoint_path_load_model_30_epochs_shuf1)
loaded_model_shuf2.load_weights(checkpoint_path_load_model_30_epochs_shuf2)
loaded_model_shuf3.load_weights(checkpoint_path_load_model_30_epochs_shuf3)

In [ ]:
optimizer = Adam(lr=1e-4)

In [ ]:
loaded_model_shuf1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
loaded_model_shuf1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
loaded_model_shuf2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
loaded_model_shuf3.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
from scipy.stats import mode

In [ ]:
pp_X_sub_466.shape

(4020, 3000, 1)

In [ ]:
pp_X_sub_466

array([[[ 0.09327535],
        [-0.18087695],
        [-0.03397775],
        ...,
        [-0.07818146],
        [-0.02995923],
        [-0.17060739]],

       [[-0.09961356],
        [-0.18891398],
        [-0.07818146],
        ...,
        [ 0.08389881],
        [-0.03888928],
        [-0.0442473 ]],

       [[-0.06746541],
        [ 0.00308192],
        [ 0.00084941],
        ...,
        [ 0.07898729],
        [-0.05853537],
        [ 0.04594613]],

       ...,

       [[ 0.10104849],
        [ 0.09743289],
        [ 0.07091844],
        ...,
        [ 0.10185196],
        [ 0.05404561],
        [ 0.02833462]],

       [[ 0.04199358],
        [ 0.05002827],
        [ 0.04520746],
        ...,
        [-0.23158733],
        [-0.20788502],
        [-0.17012202]],

       [[-0.02750641],
        [-0.10343415],
        [ 0.01949647],
        ...,
        [ 0.0162826 ],
        [ 0.09180861],
        [ 0.17135196]]])

In [ ]:
x_dt_transfer_rshp.shape

(116718, 3000, 1)

In [ ]:
x_dt_transfer_rshp

array([[[ 0.0680144 ],
        [ 0.08291509],
        [ 0.07762775],
        ...,
        [-0.07714708],
        [-0.08483776],
        [-0.09012509]],

       [[ 0.1047289 ],
        [ 0.05451331],
        [-0.02013147],
        ...,
        [-0.17123062],
        [-0.14951577],
        [-0.12146743]],

       [[ 0.00228081],
        [-0.03130934],
        [-0.00767183],
        ...,
        [ 0.07277867],
        [ 0.05536155],
        [ 0.02965057]],

       ...,

       [[ 0.01339153],
        [ 0.03071675],
        [ 0.0298049 ],
        ...,
        [ 0.041659  ],
        [ 0.03299639],
        [ 0.02433378]],

       [[-0.0574574 ],
        [-0.06562874],
        [-0.10119811],
        ...,
        [-0.07091608],
        [-0.06851275],
        [-0.09014277]],

       [[-0.06574184],
        [-0.08269832],
        [-0.08636458],
        ...,
        [-0.15052423],
        [-0.08728115],
        [-0.04282768]]])

In [ ]:
import tensorflow as tf
#tf.config.run_functions_eagerly(True)
#tf.config.experimental_run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

In [ ]:
accuracy_shuf1_model = loaded_model_shuf1.evaluate(pp_X_sub_482, y_sub_482_) #checking the model's perforamance on the subject dt

96/96 [==============================] - 11s 13ms/step - loss: 0.4133 - accuracy: 0.8678


In [ ]:
accuracy_shuf2_model = loaded_model_shuf2.evaluate(pp_X_sub_482, y_sub_482_)

In [ ]:
accuracy_shuf3_model = loaded_model_shuf3.evaluate(pp_X_sub_482, y_sub_482_)

96/96 [==============================] - 3s 12ms/step - loss: 0.4543 - accuracy: 0.8632


In [ ]:
accuracy_shuf1_transfer_set_model = loaded_model_shuf1.evaluate(x_dt_transfer_rshp, y_dt_transfer_) #checking the models performance on the transfer set

3648/3648 [==============================] - 753s 206ms/step - loss: 0.4014 - accuracy: 0.8454


In [ ]:
accuracy_shuf2_transfer_set_model = loaded_model_shuf2.evaluate(x_dt_transfer_rshp, y_dt_transfer_)

3648/3648 [==============================] - 749s 205ms/step - loss: 0.4046 - accuracy: 0.8416


In [ ]:
accuracy_shuf3_transfer_set_model = loaded_best_model_shuf3.evaluate(x_dt_transfer_rshp, y_dt_transfer_)

433/433 [==============================] - 3s 7ms/step - loss: 1.3797 - accuracy: 0.6902


In [ ]:
y_sub_466_.shape

(4020, 5)

In [ ]:
y_sub_466_

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
#adding 1 to the predictions because argmax maps labels to a number starting from 0 and not 1, will fix this in the kd+personalization experiment

## Making predictions on the transfer set and aggregating the predictions

In [ ]:
x_dt_transfer

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2960,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
2302,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,0.016102,0.014660,0.029080,0.012738,0.022351,0.043981,0.045423,0.052633,0.038213,0.029080,0.021870,0.013218,0.004566,0.002644,-0.000721,0.005047,0.007450,0.023793,0.011776,0.007450,0.001682,0.003605,0.011776,0.011296,0.008892,0.029561,0.028119,0.032445,0.031003,0.037732,...,-0.049268,-0.057440,-0.060324,-0.058401,-0.060804,-0.068014,-0.063688,-0.054556,-0.054075,-0.053114,-0.050230,-0.060324,-0.059843,-0.054075,-0.050710,-0.040136,-0.057440,-0.053594,-0.037252,-0.033406,-0.025716,-0.005047,0.004086,0.008892,0.010815,0.014660,0.031484,0.020909,0.044942,0.022832,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
31195,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,-0.013346,-0.023751,-0.005203,0.001583,0.003393,0.020584,0.017417,-0.010631,0.007012,-0.006560,0.007012,0.004750,0.039132,0.064466,0.067180,0.054513,0.048180,0.033251,0.040037,0.040037,0.025108,0.005203,0.011988,-0.031894,-0.048180,-0.108348,-0.094776,-0.113324,-0.091157,-0.108800,...,0.058585,0.043203,0.015608,0.062656,0.059942,0.095681,0.097491,0.060394,0.055870,0.040489,0.015608,0.020584,0.036870,0.050894,0.066728,0.090252,0.130515,0.178017,0.179826,0.196565,0.176659,0.162635,0.115586,0.076228,0.041846,0.000679,-0.002941,-0.046823,-0.040037,-0.070347,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
82207,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,-0.021357,-0.010160,-0.032553,-0.015136,-0.024260,-0.035042,-0.059508,-0.047482,-0.049141,-0.018869,-0.009331,-0.001866,0.004354,-0.001037,-0.005598,0.006013,0.027577,0.029651,0.064485,0.063241,0.025504,0.040433,0.035456,0.042506,0.026333,0.035456,0.023430,-0.001451,-0.027162,0.024674,...,0.003110,0.022601,0.001037,0.003525,-0.009745,-0.031724,-0.015136,-0.012233,-0.014722,-0.036286,-0.028406,-0.025918,0.001866,0.019698,0.014722,-0.031309,0.011819,-0.007257,-0.003110,0.018454,0.010575,-0.002696,0.002281,0.011404,0.044580,0.027162,0.057020,0.086464,0.056191,0.036700,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
2198,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,-0.031484,-0.035329,-0.035810,-0.031964,-0.034368,-0.031964,-0.030522,-0.035810,-0.048788,-0.046865,-0.055998,-0.069456,-0.068014,-0.070898,-0.063208,-0.052152,-0.059362,-0.066092,-0.058401,-0.045904,-0.036290,-0.023793,-0.017544,-0.019467,-0.021390,-0.020909,-0.014660,-0.023793,-0.024754,-0.026677,...,-0.033887,-0.028119,-0.022351,-0.020909,-0.029561,-0.028600,-0.027638,-0.034368,-0.039655,-0.025716,-0.023793,-0.020909,-0.009854,-0.011296,-0.015141,-0.002163,-0.003124,-0.004086,-0.016102,-0.026196,-0.025235,-0.013699,-0.003605,0.000721,0.014660,0.014180,0.032445,0.021870,0.022832,0.010334,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
107738,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,0.081305,-0.077792,0.055954,0.078245,0.012683,0.071252,0.033226,-0.018349,0.053332,0.029292,0.057265,0.113648,0.114960,-0.008733,0.083053,0.309896,-0.039329,0.017054,0.020988,0.075623,0.035849,0.084801,0.085238,0.089172,0.064259,0.090920,0.112337,0.100536,0.103158,0.123701,...,0.202375,0.048524,0.044590,0.061636,0.131569,-0.057686,-0.183127,-0.274039,-0.252622,-0.243444,-0.238636,-0.231206,-0.213723,-0.209789,-0.165644,-0.112758,-0.099208,0.284546,0.265314,0.390318,0.430966,0.370650

In [ ]:
type(x_dt_transfer)

pandas.core.frame.DataFrame

In [ ]:
for sple in x_dt_transfer:
    #sple_rshp=sple.reshape(1,3000,1)
    print(type(sple))
    print(type(np.array(sple)))

Streaming output truncated to the last 5000 lines.
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.ndarray'>

In [ ]:
preds_shuf1=[]
preds_shuf2=[]
preds_shuf3=[]

m_vote=[]

for sple in np.array(x_dt_transfer):
    sple_rshp=sple.reshape(1,3000,1)
    p1_i=loaded_model_shuf1.predict(sple_rshp)
    p1_argmx_i=np.argmax(p1_i,axis=1)
    p1_argmx_i_final=(p1_argmx_i)
    preds_shuf1.append(p1_argmx_i_final)
    p2_i=loaded_model_shuf2.predict(sple_rshp)
    p2_argmx_i=np.argmax(p2_i,axis=1)
    p2_argmx_i_final=(p2_argmx_i)
    preds_shuf2.append(p2_argmx_i_final)
    p3_i=loaded_model_shuf3.predict(sple_rshp)
    p3_argmx_i=np.argmax(p3_i,axis=1)
    p3_argmx_i_final=(p3_argmx_i)
    preds_shuf3.append(p3_argmx_i_final)
    mode_i=mode([p1_argmx_i_final,p2_argmx_i_final,p3_argmx_i_final])
    m_vote_i=mode_i[0][0]
    m_vote.append(m_vote_i)

In [ ]:
print(y_dt_transfer)

2302      4
31195     3
82207     1
2198      2
107738    0
         ..
20894     2
78249     4
57039     4
34319     2
17826     2
Name: sleep_stage, Length: 116718, dtype: int32


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_m1 = accuracy_score(y_dt_transfer, preds_shuf1)
print('Accuracy1: %f' % accuracy_m1)
accuracy_m2 = accuracy_score(y_dt_transfer, preds_shuf2)
print('Accuracy2: %f' % accuracy_m2)
accuracy_m3 = accuracy_score(y_dt_transfer, preds_shuf3)
print('Accuracy3: %f' % accuracy_m3)
accuracy_majority = accuracy_score(y_dt_transfer, m_vote)
print('Accuracy_maj: %f' % accuracy_majority)

Accuracy1: 0.845414
Accuracy2: 0.841593
Accuracy3: 0.849166
Accuracy_maj: 0.852516


In [ ]:
m_vote

[array([4]),
 array([3]),
 array([2]),
 array([2]),
 array([0]),
 array([2]),
 array([1]),
 array([2]),
 array([4]),
 array([2]),
 array([2]),
 array([0]),
 array([4]),
 array([4]),
 array([4]),
 array([2]),
 array([3]),
 array([2]),
 array([0]),
 array([2]),
 array([2]),
 array([0]),
 array([0]),
 array([2]),
 array([4]),
 array([2]),
 array([1]),
 array([0]),
 array([1]),
 array([2]),
 array([4]),
 array([0]),
 array([3]),
 array([0]),
 array([4]),
 array([2]),
 array([2]),
 array([2]),
 array([2]),
 array([2]),
 array([2]),
 array([0]),
 array([0]),
 array([2]),
 array([3]),
 array([3]),
 array([0]),
 array([3]),
 array([2]),
 array([2]),
 array([4]),
 array([4]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([1]),
 array([3]),
 array([0]),
 array([2]),
 array([4]),
 array([2]),
 array([3]),
 array([2]),
 array([1]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([2]),
 array([2]),
 array([2]),
 array([4]),
 array([4]),
 array([2]),

In [ ]:
type(m_vote[0])

numpy.ndarray

In [ ]:
fin_array=np.concatenate( m_vote, axis=0 )

In [ ]:
fin_array

array([4, 3, 2, ..., 4, 2, 2])

In [ ]:
#trans_x_bse_new_transfer.shape #x_dt_transfer is already a dataframe

(13836, 18)

In [ ]:
trans_x_bse_new_transfer_df #x_dt_transfer is already a dataframe

In [ ]:
x_dt_transfer['m_vote']=fin_array

In [ ]:
x_dt_transfer

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999,m_vote
2302,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,0.016102,0.014660,0.029080,0.012738,0.022351,0.043981,0.045423,0.052633,0.038213,0.029080,0.021870,0.013218,0.004566,0.002644,-0.000721,0.005047,0.007450,0.023793,0.011776,0.007450,0.001682,0.003605,0.011776,0.011296,0.008892,0.029561,0.028119,0.032445,0.031003,0.037732,...,-0.057440,-0.060324,-0.058401,-0.060804,-0.068014,-0.063688,-0.054556,-0.054075,-0.053114,-0.050230,-0.060324,-0.059843,-0.054075,-0.050710,-0.040136,-0.057440,-0.053594,-0.037252,-0.033406,-0.025716,-0.005047,0.004086,0.008892,0.010815,0.014660,0.031484,0.020909,0.044942,0.022832,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
31195,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,-0.013346,-0.023751,-0.005203,0.001583,0.003393,0.020584,0.017417,-0.010631,0.007012,-0.006560,0.007012,0.004750,0.039132,0.064466,0.067180,0.054513,0.048180,0.033251,0.040037,0.040037,0.025108,0.005203,0.011988,-0.031894,-0.048180,-0.108348,-0.094776,-0.113324,-0.091157,-0.108800,...,0.043203,0.015608,0.062656,0.059942,0.095681,0.097491,0.060394,0.055870,0.040489,0.015608,0.020584,0.036870,0.050894,0.066728,0.090252,0.130515,0.178017,0.179826,0.196565,0.176659,0.162635,0.115586,0.076228,0.041846,0.000679,-0.002941,-0.046823,-0.040037,-0.070347,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
82207,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,-0.021357,-0.010160,-0.032553,-0.015136,-0.024260,-0.035042,-0.059508,-0.047482,-0.049141,-0.018869,-0.009331,-0.001866,0.004354,-0.001037,-0.005598,0.006013,0.027577,0.029651,0.064485,0.063241,0.025504,0.040433,0.035456,0.042506,0.026333,0.035456,0.023430,-0.001451,-0.027162,0.024674,...,0.022601,0.001037,0.003525,-0.009745,-0.031724,-0.015136,-0.012233,-0.014722,-0.036286,-0.028406,-0.025918,0.001866,0.019698,0.014722,-0.031309,0.011819,-0.007257,-0.003110,0.018454,0.010575,-0.002696,0.002281,0.011404,0.044580,0.027162,0.057020,0.086464,0.056191,0.036700,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
2198,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,-0.031484,-0.035329,-0.035810,-0.031964,-0.034368,-0.031964,-0.030522,-0.035810,-0.048788,-0.046865,-0.055998,-0.069456,-0.068014,-0.070898,-0.063208,-0.052152,-0.059362,-0.066092,-0.058401,-0.045904,-0.036290,-0.023793,-0.017544,-0.019467,-0.021390,-0.020909,-0.014660,-0.023793,-0.024754,-0.026677,...,-0.028119,-0.022351,-0.020909,-0.029561,-0.028600,-0.027638,-0.034368,-0.039655,-0.025716,-0.023793,-0.020909,-0.009854,-0.011296,-0.015141,-0.002163,-0.003124,-0.004086,-0.016102,-0.026196,-0.025235,-0.013699,-0.003605,0.000721,0.014660,0.014180,0.032445,0.021870,0.022832,0.010334,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
107738,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,0.081305,-0.077792,0.055954,0.078245,0.012683,0.071252,0.033226,-0.018349,0.053332,0.029292,0.057265,0.113648,0.114960,-0.008733,0.083053,0.309896,-0.039329,0.017054,0.020988,0.075623,0.035849,0.084801,0.085238,0.089172,0.064259,0.090920,0.112337,0.100536,0.103158,0.123701,...,0.048524,0.044590,0.061636,0.131569,-0.057686,-0.183127,-0.274039,-0.252622,-0.243444,-0.238636,-0.231206,-0.213723,-0.209789,-0.165644,-0.112758,-0.099208,0.284546,0.265314,0.390318,0.430966,0.370650,0.395563,0.399934,0.405616,0.363657,

In [ ]:
#deleted this file from drive due to lack of space while performing kd with tri-training and personalization

In [ ]:
x_dt_transfer.to_csv(r'/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_50_subs.csv', header=False, index=False)

## KD

# Tuning the Teacher model wrt to the transfer set, its original labels

In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 98 kB 2.7 MB/s 


In [ ]:
import keras_tuner as kt

In [ ]:
teacher_30_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_30_epochs.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
optimizer=Adam(lr=1e-4)

In [ ]:
teacher_30_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
transfer_df=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_50_subs.csv',header=None)

In [ ]:
transfer_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,0.016102,0.014660,0.029080,0.012738,0.022351,0.043981,0.045423,0.052633,0.038213,0.029080,0.021870,0.013218,0.004566,0.002644,-0.000721,0.005047,0.007450,0.023793,0.011776,0.007450,0.001682,0.003605,0.011776,0.011296,0.008892,0.029561,0.028119,0.032445,0.031003,0.037732,...,-0.057440,-0.060324,-0.058401,-0.060804,-0.068014,-0.063688,-0.054556,-0.054075,-0.053114,-0.050230,-0.060324,-0.059843,-0.054075,-0.050710,-0.040136,-0.057440,-0.053594,-0.037252,-0.033406,-0.025716,-0.005047,0.004086,0.008892,0.010815,0.014660,0.031484,0.020909,0.044942,0.022832,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,-0.013346,-0.023751,-0.005203,0.001583,0.003393,0.020584,0.017417,-0.010631,0.007012,-0.006560,0.007012,0.004750,0.039132,0.064466,0.067180,0.054513,0.048180,0.033251,0.040037,0.040037,0.025108,0.005203,0.011988,-0.031894,-0.048180,-0.108348,-0.094776,-0.113324,-0.091157,-0.108800,...,0.043203,0.015608,0.062656,0.059942,0.095681,0.097491,0.060394,0.055870,0.040489,0.015608,0.020584,0.036870,0.050894,0.066728,0.090252,0.130515,0.178017,0.179826,0.196565,0.176659,0.162635,0.115586,0.076228,0.041846,0.000679,-0.002941,-0.046823,-0.040037,-0.070347,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,-0.021357,-0.010160,-0.032553,-0.015136,-0.024260,-0.035042,-0.059508,-0.047482,-0.049141,-0.018869,-0.009331,-0.001866,0.004354,-0.001037,-0.005598,0.006013,0.027577,0.029651,0.064485,0.063241,0.025504,0.040433,0.035456,0.042506,0.026333,0.035456,0.023430,-0.001451,-0.027162,0.024674,...,0.022601,0.001037,0.003525,-0.009745,-0.031724,-0.015136,-0.012233,-0.014722,-0.036286,-0.028406,-0.025918,0.001866,0.019698,0.014722,-0.031309,0.011819,-0.007257,-0.003110,0.018454,0.010575,-0.002696,0.002281,0.011404,0.044580,0.027162,0.057020,0.086464,0.056191,0.036700,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,-0.031484,-0.035329,-0.035810,-0.031964,-0.034368,-0.031964,-0.030522,-0.035810,-0.048788,-0.046865,-0.055998,-0.069456,-0.068014,-0.070898,-0.063208,-0.052152,-0.059362,-0.066092,-0.058401,-0.045904,-0.036290,-0.023793,-0.017544,-0.019467,-0.021390,-0.020909,-0.014660,-0.023793,-0.024754,-0.026677,...,-0.028119,-0.022351,-0.020909,-0.029561,-0.028600,-0.027638,-0.034368,-0.039655,-0.025716,-0.023793,-0.020909,-0.009854,-0.011296,-0.015141,-0.002163,-0.003124,-0.004086,-0.016102,-0.026196,-0.025235,-0.013699,-0.003605,0.000721,0.014660,0.014180,0.032445,0.021870,0.022832,0.010334,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,0.081305,-0.077792,0.055954,0.078245,0.012683,0.071252,0.033226,-0.018349,0.053332,0.029292,0.057265,0.113648,0.114960,-0.008733,0.083053,0.309896,-0.039329,0.017054,0.020988,0.075623,0.035849,0.084801,0.085238,0.089172,0.064259,0.090920,0.112337,0.100536,0.103158,0.123701,...,0.048524,0.044590,0.061636,0.131569,-0.057686,-0.183127,-0.274039,-0.252622,-0.243444,-0.238636,-0.231206,-0.213723,-0.209789,-0.165644,-0.112758,-0.099208,0.284546,0.265314,0.390318,0.430966,0.370650,0.395563,0.399934,0.405616,0.363657,0.385073,0.365842,0.3

In [ ]:
transfer_df[3000]

0         4
1         3
2         2
3         2
4         0
         ..
116713    2
116714    4
116715    4
116716    2
116717    2
Name: 3000, Length: 116718, dtype: int64

In [ ]:
x_transfer_set=transfer_df.drop([3000],axis=1)

In [ ]:
x_transfer_set

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2960,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,0.016102,0.014660,0.029080,0.012738,0.022351,0.043981,0.045423,0.052633,0.038213,0.029080,0.021870,0.013218,0.004566,0.002644,-0.000721,0.005047,0.007450,0.023793,0.011776,0.007450,0.001682,0.003605,0.011776,0.011296,0.008892,0.029561,0.028119,0.032445,0.031003,0.037732,...,-0.049268,-0.057440,-0.060324,-0.058401,-0.060804,-0.068014,-0.063688,-0.054556,-0.054075,-0.053114,-0.050230,-0.060324,-0.059843,-0.054075,-0.050710,-0.040136,-0.057440,-0.053594,-0.037252,-0.033406,-0.025716,-0.005047,0.004086,0.008892,0.010815,0.014660,0.031484,0.020909,0.044942,0.022832,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,-0.013346,-0.023751,-0.005203,0.001583,0.003393,0.020584,0.017417,-0.010631,0.007012,-0.006560,0.007012,0.004750,0.039132,0.064466,0.067180,0.054513,0.048180,0.033251,0.040037,0.040037,0.025108,0.005203,0.011988,-0.031894,-0.048180,-0.108348,-0.094776,-0.113324,-0.091157,-0.108800,...,0.058585,0.043203,0.015608,0.062656,0.059942,0.095681,0.097491,0.060394,0.055870,0.040489,0.015608,0.020584,0.036870,0.050894,0.066728,0.090252,0.130515,0.178017,0.179826,0.196565,0.176659,0.162635,0.115586,0.076228,0.041846,0.000679,-0.002941,-0.046823,-0.040037,-0.070347,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,-0.021357,-0.010160,-0.032553,-0.015136,-0.024260,-0.035042,-0.059508,-0.047482,-0.049141,-0.018869,-0.009331,-0.001866,0.004354,-0.001037,-0.005598,0.006013,0.027577,0.029651,0.064485,0.063241,0.025504,0.040433,0.035456,0.042506,0.026333,0.035456,0.023430,-0.001451,-0.027162,0.024674,...,0.003110,0.022601,0.001037,0.003525,-0.009745,-0.031724,-0.015136,-0.012233,-0.014722,-0.036286,-0.028406,-0.025918,0.001866,0.019698,0.014722,-0.031309,0.011819,-0.007257,-0.003110,0.018454,0.010575,-0.002696,0.002281,0.011404,0.044580,0.027162,0.057020,0.086464,0.056191,0.036700,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,-0.031484,-0.035329,-0.035810,-0.031964,-0.034368,-0.031964,-0.030522,-0.035810,-0.048788,-0.046865,-0.055998,-0.069456,-0.068014,-0.070898,-0.063208,-0.052152,-0.059362,-0.066092,-0.058401,-0.045904,-0.036290,-0.023793,-0.017544,-0.019467,-0.021390,-0.020909,-0.014660,-0.023793,-0.024754,-0.026677,...,-0.033887,-0.028119,-0.022351,-0.020909,-0.029561,-0.028600,-0.027638,-0.034368,-0.039655,-0.025716,-0.023793,-0.020909,-0.009854,-0.011296,-0.015141,-0.002163,-0.003124,-0.004086,-0.016102,-0.026196,-0.025235,-0.013699,-0.003605,0.000721,0.014660,0.014180,0.032445,0.021870,0.022832,0.010334,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,0.081305,-0.077792,0.055954,0.078245,0.012683,0.071252,0.033226,-0.018349,0.053332,0.029292,0.057265,0.113648,0.114960,-0.008733,0.083053,0.309896,-0.039329,0.017054,0.020988,0.075623,0.035849,0.084801,0.085238,0.089172,0.064259,0.090920,0.112337,0.100536,0.103158,0.123701,...,0.202375,0.048524,0.044590,0.061636,0.131569,-0.057686,-0.183127,-0.274039,-0.252622,-0.243444,-0.238636,-0.231206,-0.213723,-0.209789,-0.165644,-0.112758,-0.099208,0.284546,0.265314,0.390318,0.430966,0.370650,0.395563,0.399934,

In [ ]:
y_transfer=transfer_df[3000]

In [ ]:
y_transfer

0         4
1         3
2         2
3         2
4         0
         ..
116713    2
116714    4
116715    4
116716    2
116717    2
Name: 3000, Length: 116718, dtype: int64

In [ ]:
x_transfer_set_rshp=np.array(x_transfer_set).reshape((116718,3000,1))

In [ ]:
x_transfer_set_rshp

array([[[ 0.0680144 ],
        [ 0.08291509],
        [ 0.07762775],
        ...,
        [-0.07714708],
        [-0.08483776],
        [-0.09012509]],

       [[ 0.1047289 ],
        [ 0.05451331],
        [-0.02013147],
        ...,
        [-0.17123062],
        [-0.14951577],
        [-0.12146743]],

       [[ 0.00228081],
        [-0.03130934],
        [-0.00767183],
        ...,
        [ 0.07277867],
        [ 0.05536155],
        [ 0.02965057]],

       ...,

       [[ 0.01339153],
        [ 0.03071675],
        [ 0.0298049 ],
        ...,
        [ 0.041659  ],
        [ 0.03299639],
        [ 0.02433378]],

       [[-0.0574574 ],
        [-0.06562874],
        [-0.10119811],
        ...,
        [-0.07091608],
        [-0.06851275],
        [-0.09014277]],

       [[-0.06574184],
        [-0.08269832],
        [-0.08636458],
        ...,
        [-0.15052423],
        [-0.08728115],
        [-0.04282768]]])

In [ ]:
y_transfer_=np_utils.to_categorical(y_transfer)

In [ ]:
y_transfer_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

## Training the teacher model wrt to the transfer set, majority voted labels

In [ ]:
checkpoint_teacher_model_30_epochs = "/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs.ckpt"

In [ ]:
checkpoint_teacher_model_30_epochs = "/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_30_epochs_dir = os.path.dirname(checkpoint_teacher_model_30_epochs)

In [ ]:
cp_callback_teacher_model_30_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_30_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=30)

In [ ]:
teacher_30_epochs.fit(x_transfer_set_rshp, y_transfer_, epochs=30, callbacks=[cp_callback_teacher_model_30_epochs])

Epoch 1/30
3648/3648 [==============================] - 46s 8ms/step - loss: 0.6881 - accuracy: 0.7223
Epoch 2/30
3648/3648 [==============================] - 27s 7ms/step - loss: 0.4046 - accuracy: 0.8414
Epoch 3/30
3648/3648 [==============================] - 27s 7ms/step - loss: 0.3315 - accuracy: 0.8688
Epoch 4/30
3648/3648 [==============================] - 27s 7ms/step - loss: 0.2891 - accuracy: 0.8857
Epoch 5/30
3648/3648 [==============================] - 27s 7ms/step - loss: 0.2562 - accuracy: 0.8991
Epoch 6/30
3648/3648 [==============================] - 27s 7ms/step - loss: 0.2300 - accuracy: 0.9099
Epoch 7/30
3648/3648 [==============================] - 27s 7ms/step - loss: 0.2134 - accuracy: 0.9153
Epoch 8/30
3648/3648 [==============================] - 27s 7ms/step - loss: 0.1986 - accuracy: 0.9214
Epoch 9/30
3648/3648 [==============================] - 27s 7ms/step - loss: 0.1903 - accuracy: 0.9248
Epoch 10/30
3648/3648 [==============================] - 27s 7ms/step - l

In [ ]:
latest_teacher_30_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_30_epochs_dir)

In [ ]:
check_pt_path="/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt"

In [ ]:
print(latest_teacher_30_epochs)
print(check_pt_path)

/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt
/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt


In [ ]:
accuracy_teacher_model_30_epochs_original = teacher_30_epochs.evaluate(pp_X_sub_482, y_sub_482_) #just change the data to test data, the accuracy will change, this was the previous output with old data (complete sub data and not just test data)

96/96 [==============================] - 2s 6ms/step - loss: 0.8174 - accuracy: 0.8521


In [ ]:
teacher_30_epochs=model_b(verbose=VBS)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
teacher_30_epochs.load_weights(check_pt_path)

In [ ]:
teacher_30_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
sub_test_data=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_test_sub.csv',header=None)

In [ ]:
x_test_dt=sub_test_data.drop([3000],axis=1)
y_test_dt=sub_test_data[3000]

In [ ]:
x_test_dt.shape

(494, 3000)

In [ ]:
y_test_dt_=np_utils.to_categorical(y_test_dt)

In [ ]:
x_test_dt_rshp=np.array(x_test_dt).reshape(494,3000,1)

In [ ]:
accuracy_teacher_model_30_epochs_wts_loaded = teacher_30_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 0s 4ms/step - loss: 1.4282 - accuracy: 0.7368


## student model tuning wrt the transfer set, original labels

In [ ]:
import tensorflow as tf

In [ ]:
from keras_contrib.layers import CRF

In [ ]:
def build_model(hp):
  model = tf.keras.Sequential()
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units1 = hp.Int('units1', min_value=18, max_value=512, step=18)
  hp_units2 = hp.Int('units2', min_value=15, max_value=512, step=5)
  hp_units3 = hp.Int('units3', min_value=10, max_value=512, step=5)
  model.add(Dense(units=hp_units1, input_dim=18, activation=hp.Choice('act1', values=['relu','sigmoid','tanh'])))
  model.add(tf.keras.layers.Dense(units=hp_units2, activation=hp.Choice('act2', values=['relu','sigmoid','tanh'])))
  model.add(tf.keras.layers.Dense(units=hp_units3, activation=hp.Choice('act3', values=['sigmoid','relu','tanh'])))
  model.add(Dense(5, activation='softmax'))
  hp_learning_rate_seq = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  optimizer_seq = Adam(learning_rate=hp_learning_rate_seq)
  model.compile(optimizer=optimizer_seq, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
tuner = kt.Hyperband(
    build_model,
    objective='accuracy',
    max_epochs=30,
    hyperband_iterations=3, seed=100)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 7
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 18, 'max_value': 512, 'step': 18, 'sampling': None}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 15, 'max_value': 512, 'step': 5, 'sampling': None}
units3 (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 512, 'step': 5, 'sampling': None}
act1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
act2 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
act3 (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'relu', 'tanh'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
tuner.search(x_transfer_set_rshp, y_transfer_set_)

Trial 270 Complete [00h 00m 59s]
accuracy: 0.4689939320087433

Best accuracy So Far: 0.5725643038749695
Total elapsed time: 01h 00m 49s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='accuracy', direction='max')
Trial summary
Hyperparameters:
units1: 36
units2: 155
units3: 430
act1: relu
act2: relu
act3: relu
learning_rate: 0.001
tuner/epochs: 30
tuner/initial_epoch: 10
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: d9108e42f21af93f7b1ed22095ecf641
Score: 0.5725643038749695
Trial summary
Hyperparameters:
units1: 450
units2: 345
units3: 335
act1: relu
act2: relu
act3: relu
learning_rate: 0.001
tuner/epochs: 30
tuner/initial_epoch: 10
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: f3e9c6ecbf6b8163390f1f2756d0270c
Score: 0.5688782930374146
Trial summary
Hyperparameters:
units1: 90
units2: 100
units3: 420
act1: relu
act2: relu
act3: tanh
learning_rate: 0.001
tuner/epochs: 30
tuner/initial_epoch: 10
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 9dffb785d2df5f174563f6a88bd5849a
Score: 0.5648308992385864
Trial summary
Hyperparameters:
units1: 306
units2: 165
units3: 210
act1: relu
act2

In [ ]:
best_model = tuner.get_best_models(1)[0]

In [ ]:
best_model.get_config()

{'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 18),
    'dtype': 'float32',
    'name': 'dense_input',
    'ragged': False,
    'sparse': False}},
  {'class_name': 'Dense',
   'config': {'activation': 'relu',
    'activity_regularizer': None,
    'batch_input_shape': (None, 18),
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'dtype': 'float32',
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'kernel_regularizer': None,
    'name': 'dense',
    'trainable': True,
    'units': 36,
    'use_bias': True}},
  {'class_name': 'Dense',
   'config': {'activation': 'relu',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'dtype': 'float32',
    'kernel_constraint': None,
    'kernel_initializer': {

In [ ]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 36)                684       
                                                                 
 dense_1 (Dense)             (None, 155)               5735      
                                                                 
 dense_2 (Dense)             (None, 430)               67080     
                                                                 
 dense_3 (Dense)             (None, 5)                 2155      
                                                                 
Total params: 75,654
Trainable params: 75,654
Non-trainable params: 0
_________________________________________________________________


In [ ]:
best_learning_rate_student=0.001
optimizer_student = Adam(learning_rate=best_learning_rate_student)

In [ ]:
from tensorflow.keras.models import model_from_json
json_config_best_model_student_50_subs = best_model.to_json()
with open("/content/gdrive/My Drive/physionet/ensemble/json_config_best_model_student_50_subs.json","w") as file:
  file.write(json_config_best_model_student_50_subs)
#best_model.save("/content/gdrive/My Drive/dt_shuf1/s828/best_model_s828")

30 epochs

In [ ]:
from tensorflow.keras.models import model_from_json
with open("/content/gdrive/My Drive/physionet/ensemble/json_config_best_model_student_50_subs.json","r") as file:
  read_model=file.read()


student_30_epochs=model_from_json(read_model)

In [ ]:
student_30_epochs.compile(optimizer=optimizer_student, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint_student_model_30_epochs = "/content/gdrive/My Drive/physionet/ensemble/student/student_30_epochs/weights-30_epochs.ckpt"

In [ ]:
checkpoint_student_model_30_epochs_dir = os.path.dirname(checkpoint_student_model_30_epochs)

In [ ]:
cp_callback_student_model_30_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_student_model_30_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=30)

In [ ]:
student_30_epochs.fit(x_transfer_set_rshp, y_transfer_set_, epochs=30, callbacks=[cp_callback_student_model_30_epochs])

Epoch 1/100
433/433 [==============================] - 2s 4ms/step - loss: 1.0323 - accuracy: 0.5906
Epoch 2/100
433/433 [==============================] - 2s 4ms/step - loss: 0.9359 - accuracy: 0.6174
Epoch 3/100
433/433 [==============================] - 2s 5ms/step - loss: 0.9072 - accuracy: 0.6328
Epoch 4/100
433/433 [==============================] - 2s 4ms/step - loss: 0.8857 - accuracy: 0.6513
Epoch 5/100
433/433 [==============================] - 2s 5ms/step - loss: 0.8712 - accuracy: 0.6610
Epoch 6/100
433/433 [==============================] - 2s 5ms/step - loss: 0.8547 - accuracy: 0.6654
Epoch 7/100
433/433 [==============================] - 2s 5ms/step - loss: 0.8422 - accuracy: 0.6699
Epoch 8/100
433/433 [==============================] - 2s 5ms/step - loss: 0.8357 - accuracy: 0.6746
Epoch 9/100
433/433 [==============================] - 2s 4ms/step - loss: 0.8205 - accuracy: 0.6800
Epoch 10/100
433/433 [==============================] - 2s 4ms/step - loss: 0.8162 - accura

In [ ]:
latest_student_30_epochs=tensorflow.train.latest_checkpoint(checkpoint_student_model_30_epochs_dir)

In [ ]:
print(latest_student_30_epochs)

/content/gdrive/My Drive/ensemble/s828/student/student_100_epochs/weights-100-0.777.ckpt


In [ ]:
student_30_epochs.load_weights(latest_student_30_epochs)

In [ ]:
student_30_epochs.compile(optimizer=optimizer_student, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_student_model_30_epochs = student_30_epochs.evaluate(pp_X_sub_482, y_sub_482_)

23/23 [==============================] - 0s 4ms/step - loss: 3.5819 - accuracy: 0.6104


200 epochs

In [ ]:
from tensorflow.keras.models import model_from_json
with open("/content/gdrive/My Drive/physionet/ensemble/json_config_best_model_student_50_subs.json","r") as file:
  read_model=file.read()


student_60_epochs=model_from_json(read_model)

In [ ]:
student_60_epochs.compile(optimizer=optimizer_student, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint_student_model_60_epochs = "/content/gdrive/My Drive/physionet/ensemble/student/student_60_epochs/weights-60_epochs.ckpt"

In [ ]:
checkpoint_student_model_60_epochs_dir = os.path.dirname(checkpoint_student_model_60_epochs)

In [ ]:
cp_callback_student_model_60_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_student_model_60_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=60)

In [ ]:
student_60_epochs.fit(x_transfer_set_rshp, y_transfer_set_, epochs=60, callbacks=[cp_callback_student_model_60_epochs])

Epoch 1/200
433/433 [==============================] - 2s 5ms/step - loss: 1.0281 - accuracy: 0.5854
Epoch 2/200
433/433 [==============================] - 2s 5ms/step - loss: 0.9549 - accuracy: 0.6023
Epoch 3/200
433/433 [==============================] - 2s 5ms/step - loss: 0.9207 - accuracy: 0.6233
Epoch 4/200
433/433 [==============================] - 2s 5ms/step - loss: 0.9007 - accuracy: 0.6393
Epoch 5/200
433/433 [==============================] - 2s 4ms/step - loss: 0.8883 - accuracy: 0.6492
Epoch 6/200
433/433 [==============================] - 2s 4ms/step - loss: 0.8721 - accuracy: 0.6588
Epoch 7/200
433/433 [==============================] - 2s 5ms/step - loss: 0.8622 - accuracy: 0.6648
Epoch 8/200
433/433 [==============================] - 2s 4ms/step - loss: 0.8535 - accuracy: 0.6644
Epoch 9/200
433/433 [==============================] - 2s 5ms/step - loss: 0.8444 - accuracy: 0.6684
Epoch 10/200
433/433 [==============================] - 2s 5ms/step - loss: 0.8334 - accura

In [ ]:
latest_student_60_epochs=tensorflow.train.latest_checkpoint(checkpoint_student_model_60_epochs_dir)

In [ ]:
print(latest_student_60_epochs)

/content/gdrive/My Drive/ensemble/s828/student/student_200_epochs/weights-200-0.804.ckpt


In [ ]:
student_60_epochs.load_weights(latest_student_60_epochs)

In [ ]:
student_60_epochs.compile(optimizer=optimizer_student, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_student_model_60_epochs = student_200_epochs.evaluate(pp_X_sub_482, y_sub_482_)

23/23 [==============================] - 0s 4ms/step - loss: 3.2923 - accuracy: 0.5513


300 epochs

In [ ]:
from tensorflow.keras.models import model_from_json
with open("/content/gdrive/My Drive/ensemble/s828/json_config_best_model_student_s828.json","r") as file:
  read_model=file.read()


student_300_epochs=model_from_json(read_model)

In [ ]:
student_300_epochs.compile(optimizer=optimizer_student, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint_student_model_300_epochs = "/content/gdrive/My Drive/ensemble/s828/student/student_300_epochs/weights-{epoch:02d}-{accuracy:.3f}.ckpt"

In [ ]:
checkpoint_student_model_300_epochs_dir = os.path.dirname(checkpoint_student_model_300_epochs)

In [ ]:
cp_callback_student_model_300_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_student_model_300_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=300)

In [ ]:
student_300_epochs.fit(x_transfer_set_cp_ar, y_transfer_ar_cp_ohe, epochs=300, callbacks=[cp_callback_student_model_300_epochs])

Epoch 1/300
433/433 [==============================] - 2s 4ms/step - loss: 1.0121 - accuracy: 0.5907
Epoch 2/300
433/433 [==============================] - 2s 4ms/step - loss: 0.9434 - accuracy: 0.6094
Epoch 3/300
433/433 [==============================] - 2s 4ms/step - loss: 0.9051 - accuracy: 0.6378
Epoch 4/300
433/433 [==============================] - 2s 4ms/step - loss: 0.8865 - accuracy: 0.6465
Epoch 5/300
433/433 [==============================] - 2s 4ms/step - loss: 0.8674 - accuracy: 0.6599
Epoch 6/300
433/433 [==============================] - 2s 4ms/step - loss: 0.8530 - accuracy: 0.6678
Epoch 7/300
433/433 [==============================] - 2s 4ms/step - loss: 0.8472 - accuracy: 0.6698
Epoch 8/300
433/433 [==============================] - 2s 4ms/step - loss: 0.8397 - accuracy: 0.6797
Epoch 9/300
433/433 [==============================] - 2s 4ms/step - loss: 0.8306 - accuracy: 0.6810
Epoch 10/300
433/433 [==============================] - 2s 4ms/step - loss: 0.8212 - accura

In [ ]:
latest_student_300_epochs=tensorflow.train.latest_checkpoint(checkpoint_student_model_300_epochs_dir)

In [ ]:
print(latest_student_300_epochs)

/content/gdrive/My Drive/ensemble/s828/student/student_300_epochs/weights-300-0.847.ckpt


In [ ]:
student_300_epochs.load_weights(latest_student_300_epochs)

In [ ]:
student_300_epochs.compile(optimizer=optimizer_student, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_student_model_300_epochs = student_300_epochs.evaluate(trans_x_train_s828_cp, y_train_s828_ar_ohe)

23/23 [==============================] - 0s 4ms/step - loss: 5.9051 - accuracy: 0.6442


400 epochs

In [ ]:
from tensorflow.keras.models import model_from_json
with open("/content/gdrive/My Drive/ensemble/s828/json_config_best_model_student_s828.json","r") as file:
  read_model=file.read()


student_400_epochs=model_from_json(read_model)

In [ ]:
student_400_epochs.compile(optimizer=optimizer_student, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint_student_model_400_epochs = "/content/gdrive/My Drive/ensemble/s828/student/student_400_epochs/weights-{epoch:02d}-{accuracy:.3f}.ckpt"

In [ ]:
checkpoint_student_model_400_epochs_dir = os.path.dirname(checkpoint_student_model_400_epochs)

In [ ]:
cp_callback_student_model_400_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_student_model_400_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=400)

In [ ]:
student_400_epochs.fit(x_transfer_set_cp_ar, y_transfer_ar_cp_ohe, epochs=400, callbacks=[cp_callback_student_model_400_epochs])

Epoch 1/400
433/433 [==============================] - 2s 5ms/step - loss: 1.0191 - accuracy: 0.5879
Epoch 2/400
433/433 [==============================] - 2s 5ms/step - loss: 0.9466 - accuracy: 0.6043
Epoch 3/400
433/433 [==============================] - 2s 4ms/step - loss: 0.9135 - accuracy: 0.6316
Epoch 4/400
433/433 [==============================] - 2s 4ms/step - loss: 0.8923 - accuracy: 0.6429
Epoch 5/400
433/433 [==============================] - 2s 5ms/step - loss: 0.8780 - accuracy: 0.6556
Epoch 6/400
433/433 [==============================] - 2s 4ms/step - loss: 0.8654 - accuracy: 0.6586
Epoch 7/400
433/433 [==============================] - 2s 4ms/step - loss: 0.8524 - accuracy: 0.6672
Epoch 8/400
433/433 [==============================] - 2s 4ms/step - loss: 0.8459 - accuracy: 0.6701
Epoch 9/400
433/433 [==============================] - 2s 5ms/step - loss: 0.8349 - accuracy: 0.6781
Epoch 10/400
433/433 [==============================] - 2s 5ms/step - loss: 0.8272 - accura

In [ ]:
latest_student_400_epochs=tensorflow.train.latest_checkpoint(checkpoint_student_model_400_epochs_dir)

In [ ]:
print(latest_student_400_epochs)

/content/gdrive/My Drive/ensemble/s828/student/student_400_epochs/weights-400-0.863.ckpt


In [ ]:
student_400_epochs.load_weights(latest_student_400_epochs)

In [ ]:
student_400_epochs.compile(optimizer=optimizer_student, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_student_model_400_epochs = student_400_epochs.evaluate(trans_x_train_s828_cp, y_train_s828_ar_ohe)

23/23 [==============================] - 0s 4ms/step - loss: 7.4530 - accuracy: 0.6090


KD with best heavy wt lstm model

In [ ]:
import keras

In [ ]:
from tensorflow.keras.models import model_from_json
with open("/content/gdrive/My Drive/ensemble/s828/json_config_best_model_student_s828.json","r") as file:
  read_model=file.read()


student_model=model_from_json(read_model)

In [ ]:
teacher_model=model_b(verbose=VBS)

In [ ]:
#best_learning_rate_teacher=0.001
#optimizer_teacher = Adam(learning_rate=best_learning_rate_teacher)

In [ ]:
checkpoint_teacher = "/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs.ckpt"

In [ ]:
checkpoint_teacher_dir = os.path.dirname(checkpoint_teacher)

In [ ]:
#latest_teacher=tensorflow.train.latest_checkpoint(checkpoint_teacher_dir)

In [ ]:
print(checkpoint_teacher)

/content/gdrive/My Drive/ensemble/s828/teacher_300_epochs/weights-300-0.971.ckpt


In [ ]:
teacher_model.load_weights(checkpoint_teacher)

In [ ]:
teacher_model.compile(optimizer=optimizer_teacher, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 18, 1)]      0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 18, 32)       3232        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 18, 32)       25632       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 9, 32)        0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
accuracy_best_teacher = teacher_model.evaluate(pp_X_sub_482, y_sub_482_)

23/23 [==============================] - 2s 7ms/step - loss: 4.8575 - accuracy: 0.6231


In [ ]:

class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
checkpoint_distiller_30_epochs = "/content/gdrive/My Drive/physionet/ensemble/distilled_model/50_subs/distilled_model_30_epochs/weights-30_epochs.ckpt"

In [ ]:
checkpoint_distiller_30_epochs_dir = os.path.dirname(checkpoint_distiller_30_epochs)

In [ ]:
cp_callback_distiller_30_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_distiller_30_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=30)

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=4,
)

# Distill teacher to student
distiller.fit(x_transfer_set_rshp, y_transfer_set_, epochs=30)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/400


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


433/433 [==============================] - 7s 10ms/step - categorical_accuracy: 0.5859 - student_loss: 1.0411 - distillation_loss: 0.0017
Epoch 2/400
433/433 [==============================] - 3s 8ms/step - categorical_accuracy: 0.6156 - student_loss: 0.9411 - distillation_loss: 0.0020
Epoch 3/400
433/433 [==============================] - 3s 8ms/step - categorical_accuracy: 0.6382 - student_loss: 0.9050 - distillation_loss: 0.0021
Epoch 4/400
433/433 [==============================] - 3s 8ms/step - categorical_accuracy: 0.6474 - student_loss: 0.8881 - distillation_loss: 0.0022
Epoch 5/400
433/433 [==============================] - 3s 8ms/step - categorical_accuracy: 0.6609 - student_loss: 0.8673 - distillation_loss: 0.0022
Epoch 6/400
433/433 [==============================] - 3s 8ms/step - categorical_accuracy: 0.6628 - student_loss: 0.8582 - distillation_loss: 0.0023
Epoch 7/400
433/433 [==============================] - 3s 8ms/step - categorical_accuracy: 0.6723 - student_loss: 0.8

In [ ]:
accuracy_distiller = distiller.evaluate(pp_X_sub_482, y_sub_482_)

23/23 [==============================] - 0s 4ms/step - categorical_accuracy: 0.6048 - student_loss: 6.3767


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


used the best performing teacher with accuracy of 62.32 on subject data;

alpha=0.1

temp=10

100 epochs: ~ 55

200 epochs: 60.48

300 epochs: 57.24

400 epochs: 59.07

500 epochs: 55.56

alpha=0.9

temp=4

100 epochs: 60.90

200 epochs: 62.17

300 epochs: 62.45

400 epochs: 60.48

500 epochs: 

